# Task 3- Quanvolutional Neural Networks

In this notebook, we follow the tutorial on Quanvolutional Neural Networks using Pennylane([here](https://pennylane.ai/qml/demos/tutorial_quanvolution/)) and explain what we have learnt, through comments within each code block as well as a brief summary in the end of the notebook.

__Contributors:__ Nandan Patel, Phalak Bhatnagar

## Quanvolutional Neural Networks

The idea behind a classical CNN is that a kernel/filter is applied to an input image to extract its important features.
The kernel is applied to different sub-images within the input image. The relevant features of the image are extracted and a new image is formed by collecting the features of these sub-images.


Steps in a QNN:
 - The pixels of our sub-image are encoded in a variational circuit
 - A unitary U is then applied over the sub-image(it could be a pre-defined unitary, could also be generated randomly)
 - The system is then measured and expectation values are calculated
 - We map the expectation values to the pixels of the output which are then mapped onto a resulting image 
 - Similarly, we can create a full output image using the calculated expectation values
 - We can perform further operations on the output image using classical/quantum layers

In [ ]:
# Importing libraries
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
# Defining the hyperparameter values
n_epochs = 30   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 50    # Size of the train dataset
n_test = 30     # Size of the test dataset

SAVE_PATH = r"C:\Users\dpsso\Desktop\QNN"  # Data saving folder
PREPROCESS = True           # If False, skip quantum processing and load data from SAVE_PATH
np.random.seed(0)           # Seed for NumPy random number generator
tf.random.set_seed(0)       # Seed for TensorFlow random number generator



In [ ]:
# Loading the dataset
mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

# Reduce dataset size
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]
test_images = test_images[:n_test]
test_labels = test_labels[:n_test]

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

# Add extra dimension for convolution channels
train_images = np.array(train_images[..., tf.newaxis], requires_grad=False)
test_images = np.array(test_images[..., tf.newaxis], requires_grad=False)


In [ ]:
## Defining the quantum circuit, this circuit is the unitary U that is then used as a kernel
dev = qml.device("default.qubit", wires=4)
# Random circuit parameters
rand_params = np.random.uniform(high=2 * np.pi, size=(n_layers, 4))

@qml.qnode(dev)
def circuit(phi):
    # Encoding of 4 classical input values becuase the sub-image is of size 2x2
    for j in range(4):
        qml.RY(np.pi * phi[j], wires=j)

    # Random quantum circuit
    RandomLayers(rand_params, wires=list(range(4)))

    # Measurement producing 4 classical output values, these 4 values are then projected onto 4 pixels of the output image
    return [qml.expval(qml.PauliZ(j)) for j in range(4)]

In [ ]:
## The kernel is projected over the 4 squares of the input pixels and the output pixel values are assigned the expectation values
def quanv(image):
   """Convolves the input image with many applications of the same quantum circuit."""
   out = np.zeros((14, 14, 4))

   # Loop over the coordinates of the top-left pixel of 2X2 squares
   for j in range(0, 28, 2):
       for k in range(0, 28, 2):
           # Process a squared 2x2 region of the image with a quantum circuit
           q_results = circuit(
               [
                   image[j, k, 0],
                   image[j, k + 1, 0],
                   image[j + 1, k, 0],
                   image[j + 1, k + 1, 0]
               ]
           )
           # Assign expectation values to different channels of the output pixel (j/2, k/2)
           for c in range(4):
               out[j // 2, k // 2, c] = q_results[c]
   return out


# # We apply this the above part to perform pre-preprocessing as it is a one-time thing, it won't consume much resources
# On the other hand, if we apply a quantum circuit to every step of image processing, it would be computationally expensive and inefficient


In [ ]:
# Splitting the data into training and testing images
if PREPROCESS == True:
    q_train_images = []
    print("Quantum pre-processing of train images:")
    for idx, img in enumerate(train_images):
        print("{}/{}        ".format(idx + 1, n_train), end="\r")
        q_train_images.append(quanv(img))
    q_train_images = np.asarray(q_train_images)

    q_test_images = []
    print("\nQuantum pre-processing of test images:")
    for idx, img in enumerate(test_images):
        print("{}/{}        ".format(idx + 1, n_test), end="\r")
        q_test_images.append(quanv(img))
    q_test_images = np.asarray(q_test_images)

    # Save pre-processed images
    np.save(SAVE_PATH + "q_train_images.npy", q_train_images)
    np.save(SAVE_PATH + "q_test_images.npy", q_test_images)


# Load pre-processed images
q_train_images = np.load(SAVE_PATH + "q_train_images.npy")
q_test_images = np.load(SAVE_PATH + "q_test_images.npy")


In [ ]:
# Applying the convolution to the input image and getting 4 output channels, these output channels would then be mapped onto the output image pixels
n_samples = 4
n_channels = 4
fig, axes = plt.subplots(1 + n_channels, n_samples, figsize=(10, 10))
for k in range(n_samples):
    axes[0, 0].set_ylabel("Input")
    if k != 0:
        axes[0, k].yaxis.set_visible(False)
    axes[0, k].imshow(train_images[k, :, :, 0], cmap="gray")

    # Plot all output channels
    for c in range(n_channels):
        axes[c + 1, 0].set_ylabel("Output [ch. {}]".format(c))
        if k != 0:
            axes[c, k].yaxis.set_visible(False)
        axes[c + 1, k].imshow(q_train_images[k, :, :, c], cmap="gray")

plt.tight_layout()
plt.show()

In [ ]:
## Classical Image Processing
## Now the quantum part is over, we take the pre-processed images and classify them into 10 different channels
def MyModel():
    """Initializes and returns a custom Keras model
    which is ready to be trained."""
    model = keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(10, activation="softmax")
    ])

    model.compile(
        optimizer='adam',
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [ ]:
# Training the model based on the pre-processed images from the quantum convolutional layer
q_model = MyModel()

q_history = q_model.fit(
    q_train_images,
    train_labels,
    validation_data=(q_test_images, test_labels),
    batch_size=4,
    epochs=n_epochs,
    verbose=2,
)

In [ ]:
## In order to compare, we skip the quantum pre-processing part and directly train the model
# Therefore, we compare classical vs quantum CNNs
# Initializing the model with a classical instance
c_model = MyModel()

c_history = c_model.fit(
    train_images,
    train_labels,
    validation_data=(test_images, test_labels),
    batch_size=4,
    epochs=n_epochs,
    verbose=2,
)

In [ ]:
## Comparing the results
import matplotlib.pyplot as plt

plt.style.use("seaborn")
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 9))

ax1.plot(q_history.history["val_accuracy"], "-ob", label="With quantum layer")
ax1.plot(c_history.history["val_accuracy"], "-og", label="Without quantum layer")
ax1.set_ylabel("Accuracy")
ax1.set_ylim([0, 1])
ax1.set_xlabel("Epoch")
ax1.legend()

ax2.plot(q_history.history["val_loss"], "-ob", label="With quantum layer")
ax2.plot(c_history.history["val_loss"], "-og", label="Without quantum layer")
ax2.set_ylabel("Loss")
ax2.set_ylim(top=2.5)
ax2.set_xlabel("Epoch")
ax2.legend()
plt.tight_layout()
plt.show()


# # We see that even though the accuracy of the classical model is high, due to the absence of pre-processing, it costs more to evaluate and compute.
# Therefore, QCNNs lower the cost while maintaing the same level of accuracy.


### Summary
We learned how to combine quantum circuits with classical convolutional neural networks. We explored the concept of a quanvolutional layer, where quantum circuits process input data to extract features. We implemented this using PennyLane and TensorFlow, preparing the dataset, designing the quanvolutional layer, and integrating it into a neural network. We also learned to train and evaluate the model on a classical machine learning task, demonstrating the potential of quantum-enhanced feature extraction.